## Original Wiki doc json

In [1]:
import json
import pprint

In [2]:
with open('data/wikipedia_documents.json') as f:
    wiki = json.load(f)

In [5]:
gen_model = 'gogamza/kobart-base-v2'

In [6]:
total_wiki_passages = [doc['text'] for doc in wiki.values()]

In [7]:
len(total_wiki_passages)

60613

In [8]:
unique_wiki_passages = list(set(total_wiki_passages))
len(unique_wiki_passages)

56737

In [9]:
# unique_wiki_json
unique_wiki_dict = {idx: passage for idx, passage in enumerate(unique_wiki_passages)}

In [11]:
# save
with open('unique_wiki_passages.json', 'w') as f:
    json.dump(unique_wiki_dict, f, ensure_ascii=False, indent=4)

## Chunking

In [18]:
# # passage를 \n\n 간격으로하여 하나의 txt 파일로 저장
# with open('unique_wiki_passages.txt', 'w') as f:
#     f.write('\n\n'.join(unique_wiki_passages))

In [12]:
# # load
# with open('unique_wiki_passages.json') as f:
#     unique_wiki_dict_sample = json.load(f)

In [19]:
# 간단 길이 분포 확인
import pandas as pd
s = pd.Series(unique_wiki_passages)
s.str.len().describe()

count    56737.000000
mean       750.820593
std        698.576330
min        184.000000
25%        416.000000
50%        578.000000
75%        856.000000
max      46099.000000
dtype: float64

In [21]:
# langchain을 활용해 tiktoken(OpenAI의 fast BPE tokenizer)로 chunk 분리
from langchain.text_splitter import RecursiveCharacterTextSplitter # character의 개수 단위 분리

chunk_size = 300
chunk_overlap = 50
with open("./unique_wiki_passages.txt") as f:
    total_wiki = f.read()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)
texts = text_splitter.split_text(total_wiki)

In [22]:
len(texts)

384363

In [26]:
texts[:10]

['1623년 3월 13일 반정이 성공하고 광해군이 인목대비의 폐위교서 발표 후 인조가 즉위하였으며, 주변에서 호위하였다. 인조 즉위 직후 광해군 폐출과 정변을 지원한 공로로 분충찬모입기명륜정사공신(奮忠贊謨立紀明倫靖社功臣) 1등에 녹훈되고, 이후',
 '1등에 녹훈되고, 이후 호조판서에 승진되고 완풍군(完豊君)에 봉군되었다. 호조판서가 되어서는 호조의 사무가 매우 많고 어지러웠으나, 엄하게 다스리고 해결하니 교활한 아전이 잔머리를 굴리지 못하였다.',
 '1624년(인조 2) 경기도관찰사(京畿道觀察使)가 되었다가 1624년(인조 2) 논공행상에 불만을 품은 이괄(李适), 한명련 등이 거병을 일으켜, 반란이 발생하자 원수부 부원수(副元帥)로서 출정, 이괄군과 교전하였다. 그러나 개성까지 이괄군을 추격했지만 여건이 되지',
 '이괄군을 추격했지만 여건이 되지 않아 더 나가지 못하고 지체했는데, 오히려 밤에 이괄군의 기습을 당하기도 했다. 더 나가지 못해, 반란군을 추격하다가 중간에 멈춘 죄로 파직되었다.',
 '곧 인조의 명으로 백의종군의 명이 내려졌으나 얼마 뒤 인조가 특별히 용서하여 다시 서용되고, 완풍군에 봉해졌다. 같은 해 어영대장(御營大將)이 되어 수어사(守禦使)를 겸하였고, 그 해 한성과 경기도의 군사 지휘를 목적으로 사직동(社稷洞) 북쪽에 총융청이',
 '사직동(社稷洞) 북쪽에 총융청이 설치되자, 겸 총융청사(摠戎廳使)가 되어 남양(南陽), 수원부, 장단군 등 3진(鎭)의 지휘와 동원권을 행사하였다.',
 '이때 대신 경기도는 나라의 인후(咽喉, 코)와 같아서 군사가 정예하지 못하면 위급한 시기를 당하여 득력(得力)하기 어렵다며 그를 총융사로 추천, 겸직하게 하였다. 동시에 훈련원의 교련(敎鍊)도 같이 맡아보았다. 이때 군율을 엄하게 정하고, 장교를',
 '이때 군율을 엄하게 정하고, 장교를 선발하여 수시로 군사훈련을 시키고, 봄가을로 크게 사열하여 상벌을 주었다. 또한 둔전(屯田)을 설치하고, 군량미를 저축하여 병농(兵農)이 서로 침탈하지 않게 하였

In [28]:
# list to dict
texts_dict = {idx: passage for idx, passage in enumerate(texts)}

In [30]:
# save
with open('unique_wiki_passages_chunked.json', 'w') as f:
    json.dump(texts_dict, f, ensure_ascii=False, indent=4)